# 0. Imports

In [1]:
import random

import numpy as np
import pandas as pd

from extrucal.extrusion import throughput_cal

from sklearn.utils import shuffle

random.seed(0)

# 1. Dataset Generation using `extrucal`

## 1.1 Preparation of Lists for Arguments

In [2]:
extruder_size = [size for size in range(20, 251, 10)]
extruder_size_df = pd.DataFrame({"extruder_size": extruder_size})
len(extruder_size)

24

In [3]:
metering_depth_percent = [
    np.round(percent, 2) for percent in np.arange(0.02, 0.1, 0.01)
]
metering_depth_percent_df = pd.DataFrame(
    {"metering_depth_percent": metering_depth_percent}
)
len(metering_depth_percent)

8

In [4]:
polymer_density = [density for density in range(800, 1500, 100)]
polymer_density_df = pd.DataFrame({"polymer_density": polymer_density})
len(polymer_density)

7

In [5]:
screw_pitch_percent = [np.round(pitch, 2) for pitch in np.arange(0.6, 2, 0.2)]
screw_pitch_percent_df = pd.DataFrame({"screw_pitch_percent": screw_pitch_percent})
len(screw_pitch_percent)

7

In [6]:
flight_width_percent = [np.round(percent, 2) for percent in np.arange(0.06, 0.2, 0.02)]
flight_width_percent_df = pd.DataFrame({"flight_width_percent": flight_width_percent})
len(flight_width_percent)

8

In [7]:
number_flight = [1, 2]
number_flight_df = pd.DataFrame({"number_flight": number_flight})
len(number_flight)

2

In [8]:
rpm = [r for r in range(1, 101, 10)]
rpm_df = pd.DataFrame({"rpm": rpm})
len(rpm)

10

## 1.2. Preparation of Dataframe

In [9]:
df = (
    extruder_size_df.join(metering_depth_percent_df, how="cross")
    .join(polymer_density_df, how="cross")
    .join(screw_pitch_percent_df, how="cross")
    .join(flight_width_percent_df, how="cross")
    .join(number_flight_df, how="cross")
    .join(rpm_df, how="cross")
)

In [10]:
len(df)

1505280

In [11]:
df["metering_depth"] = df["extruder_size"] * df["metering_depth_percent"]
df["screw_pitch"] = df["extruder_size"] * df["screw_pitch_percent"]
df["flight_width"] = df["extruder_size"] * df["flight_width_percent"]

In [12]:
new_col_order = [
    "extruder_size",
    "metering_depth",
    "polymer_density",
    "rpm",
    "screw_pitch",
    "flight_width",
    "number_flight",
]

df = df[new_col_order]
df.head()

,extruder_size,metering_depth,polymer_density,rpm,screw_pitch,flight_width,number_flight
0,20,0.4,800,1,12.0,1.2,1
1,20,0.4,800,11,12.0,1.2,1
2,20,0.4,800,21,12.0,1.2,1
3,20,0.4,800,31,12.0,1.2,1
4,20,0.4,800,41,12.0,1.2,1


In [13]:
%%time

df.loc[:, "throughput"] = df.apply(
    lambda row: throughput_cal(
        row["extruder_size"],
        row["metering_depth"],
        row["polymer_density"],
        row["rpm"],
        row["screw_pitch"],
        row["flight_width"],
        int(row["number_flight"]),
    )*random.uniform(0.95, 1.05),  # +/- 5% randomness was added to the calculated throughput
    axis=1,
)

CPU times: total: 8min 15s
Wall time: 8min 15s


In [14]:
df

,extruder_size,metering_depth,polymer_density,rpm,screw_pitch,flight_width,number_flight,throughput
0,20,0.4,800,1,12.0,1.2,1,0.010344
1,20,0.4,800,11,12.0,1.2,1,0.071806
2,20,0.4,800,21,12.0,1.2,1,0.128967
3,20,0.4,800,31,12.0,1.2,1,0.185419
4,20,0.4,800,41,12.0,1.2,1,0.250282
...,...,...,...,...,...,...,...,...
1505275,250,22.5,1400,51,450.0,50.0,2,8322.007281
1505276,250,22.5,1400,61,450.0,50.0,2,10012.364039
1505277,250,22.5,1400,71,450.0,50.0,2,11483.208191
1505278,250,22.5,1400,81,450.0,50.0,2,13878.259350


In [15]:
df = shuffle(df)

In [16]:
df

,extruder_size,metering_depth,polymer_density,rpm,screw_pitch,flight_width,number_flight,throughput
1410090,240,12.0,1400,1,144.0,14.4,2,37.513281
1349859,230,13.8,900,91,184.0,32.2,2,2438.274330
1122742,190,17.1,900,21,228.0,15.2,1,1041.914244
961430,170,6.8,1200,1,170.0,34.0,2,10.273171
271524,60,2.4,1200,41,72.0,3.6,1,37.045367
...,...,...,...,...,...,...,...,...
617581,110,8.8,1300,11,110.0,22.0,1,99.018883
585236,110,4.4,1200,61,132.0,17.6,2,247.563530
519776,100,4.0,1000,61,60.0,14.0,2,57.198810
669739,120,8.4,1000,91,216.0,21.6,2,1002.149759


## 1.3. Save as CSV file

In [17]:
df.to_csv("../data/dataset_generated_by_extrucal_from_1_rpm.csv", index=False)